# Dots Image Segmenter

This notebook is adapted from *Allen Cell Structure Segmenter* for segmentation of dot-like structures.

The purpose of this notebook is to segment a batch of timelapses videos and output .tiff files for each segmented time frame for each video file. The output will include a folder (that you will need to first create) of segmented images corresponding to each time point of a given input video and outputs for all videos will be stored within the same folder.

J. Chen, L. Ding, M.P. Viana, M.C. Hendershott, R. Yang, I.A. Mueller, S.M. Rafelski. The Allen Cell Structure Segmenter: a new open source toolkit for segmenting 3D intracellular structures in fluorescence microscopy images. bioRxiv. 2018 Jan 1:491035.

<span style='color:red'> **1. First run the two following cells then follow the red instruction lines through this notebook** </span>

In [1]:
import numpy as np
import scipy
import skimage.io
import nbimporter
import skimage
from skimage.measure import label, regionprops
import math
import glob
import os

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d
from skimage.morphology import remove_small_objects, watershed, dilation, erosion, ball, label     # function for post-processing (size filter)
from skimage.feature import peak_local_max
from skimage.measure import label
from scipy.ndimage import distance_transform_edt

In [2]:
def ConvertToHyperstack(FILE_NAME, nbChannels, nbZplanes):
    """
    A function that converts raw timelapse files into files where images are organized  into hyperstacks.
    Since the files obtained from metamorph contain no metadata and are composed simply of an array of images, the
    images need to be organized into their respective stacks.
    
    INPUTS:
       FILE_NAME: Full directory of raw (directly from Metamorph) timelapse video
       nbChannels: The number of unique colour channels in timelapse videos (must be the same for all files)
       nbZplanes: The number of planes in each stack
    
    RETURNS the image but organized as a hyperstack in order of time, channel, plane, y and x
    """
    
    raw_im = skimage.io.imread(FILE_NAME)
    
    # Find and parse dimensions of input image
    dims = raw_im.shape

    if len(dims) != 3:
        raise RuntimeError(f"Expected 3 dimensions, found {len(dims)}")

    x = dims[2]
    y = dims[1]
    zt = dims[0]

    # Calculate number of t frames (assume input image can have any number of time frames)
    t = int(zt/(nbZplanes*nbChannels))

    # Reshape to TCZYX
    im2 = raw_im.reshape((nbChannels,nbZplanes,t,y,x), order='F')
    # Move indices from CZTYX to TCZYX to match AICS format
    IMG = np.moveaxis(im2, [0, 1, 2, 3, 4], [1, 2, 0, 3, 4] )
    #IMG = np.moveaxis(im2, [0, 1, 2, 3, 4], [0, 2, 3, 4, 1])

    
    # Change type from uint16 to float32
    IMG = IMG.astype(np.float32)
    
    return IMG

<span style='color:red'> **2. Adjust parameters as desired for time series. Parameters are found using `dots_parameter_playground_SZ` Jupyter Notebook. They are presently adjusted for nucleoli in the second cell and then for nuclei in the following cell. Only change the parameters for the nucleoli cell.** </span>

<span style='color:red'> **3. In the following cell, change the variables `directoryIN` and `directoryOUT` to the appropriate directories.** </span>

`directoryIN` is the directory to the folder that contains the raw tif files that you want to segment. This directory should end with `seg_batch_input\\*`

`directoryOUT1`is the directory to the file you have created where you want the segmented images of nucleoli to be saved. It
is the directory to the `seg_batch_output_nucleoli` folder

`directoryOUT2`is the directory to the file you have created where you want the segmented images of nuclei to be saved. It
is the directory to the `seg_batch_output_nuclei` folder

All three directories must be in between quotation marks but only `directoryIN` ends with a `\\*`.

<span style='color:red'> **4. Run the next 3 cells** </span>


In [3]:
directoryIN = 'E:\\Weber Lab\\Practice Coding\\seg_batch_input\\*'
start = len(directoryIN) - 1
directoryOUT1 = 'E:\\Weber Lab\\Practice Coding\\seg_batch_output_nucleoli\\'
directoryOUT2 = 'E:\\Weber Lab\\Practice Coding\\seg_batch_output_nuclei\\'

In [4]:
#FOR NUCLEOLI
"""
 This cell extracts and segment only the images in the first channel imaged. The parameters are set to segment only nucleoli.
 Once segmented, the stack for each timepoint of each timelapse video will be stored into the seg_batch_output_centrosome 
 folder.
"""

for file_name in glob.glob(directoryIN, recursive=True):
    # Create the directory where we save the output files
    if not os.path.exists(directoryOUT1+str(file_name[start:-4])):
        os.makedirs(directoryOUT1+str(file_name[start:-4]))
    
    IMG = ConvertToHyperstack(file_name, 2, 25)
    
    # parameters
    intensity_scaling_param = [1,15]
    gaussian_smoothing_sigma = 1
    s3_param = [[0.9, 0.09]]
    minArea = 8
    structure_channel = 0
    
    ################################
    for t in range(0,IMG.shape[0]):
        struct_img0 = IMG[t,structure_channel,:,:,:].copy()
    # intensity normalization
        struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

    # smoothing with gaussian filter
        structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

    # core algorithm
        bw = dot_3d_wrapper(structure_img_smooth, s3_param)

    # post processing
        seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)
    
    #label objects with unique integers
        label = skimage.morphology.label(seg)
        
    # output
        if t<10:
            out=label.astype(np.uint8)
            writer = OmeTiffWriter()
            writer.save(out, directoryOUT1 +str(file_name[start:-4])+"\\"+'0'+str(t)+'.tiff')
        else:
            out=label.astype(np.uint8)
            writer = OmeTiffWriter()
            writer.save(out, directoryOUT1 +str(file_name[start:-4])+"\\"+str(t)+'.tiff')


In [5]:
#FOR NUCLEI
"""
 This cell extracts and segment only the images in the first channel imaged. The parameters are set to segment only the nuclei.
 Once segmented, the nuclei images for each timepoint of each timelapse video will be stored into the seg_batch_output_nucleus 
 folder
"""

for file_name in glob.glob(directoryIN, recursive=True):
    IMG = ConvertToHyperstack(file_name, 2, 25)
    
    # Create the directory where we save the output files
    if not os.path.exists(directoryOUT2+str(file_name[start:-4])):
        os.makedirs(directoryOUT2+str(file_name[start:-4]))
    
    # parameters
    intensity_scaling_param = [1,4.4]
    gaussian_smoothing_sigma = 1.5
    s3_param = [[7, 0.07]]
    minArea = 1500
    structure_channel = 0
    
    ################################
    for t in range(0,IMG.shape[0]):
        struct_img0 = IMG[t,structure_channel,:,:,:].copy()
    # intensity normalization
        struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

    # smoothing with gaussian filter
        structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

    # core algorithm
        bw = dot_3d_wrapper(structure_img_smooth, s3_param)

    # post processing
        seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)
    
    #label objects with unique integers
        label = skimage.morphology.label(seg)

    # output
        if t<10:
            out=label.astype(np.uint8)
            writer = OmeTiffWriter()
            writer.save(out, directoryOUT2 +str(file_name[start:-4])+"\\"+'0'+str(t)+'.tiff')
        else:
            out=label.astype(np.uint8)
            writer = OmeTiffWriter()
            writer.save(out, directoryOUT2 +str(file_name[start:-4])+"\\"+str(t)+'.tiff')
